In [1]:
import numpy as np
import theano
import theano.tensor as T
import time

import lasagne as L

import sys
sys.path.insert(0, '../rnn_ex/')

from HRED import HRED
from mt_load import load_mt, get_mt_voc, get_w2v_embs

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 780 (CNMeM is enabled with initial size: 30.0% of memory, cuDNN 5105)


In [2]:
# remember, now the pad value is the same as the <utt_end> token

pad_value = -1 # <utt_end>'s vector is the last one

In [3]:
mt_path = "/pio/data/data/mtriples/"
# mt_path = "/home/maciek/Desktop/mgr/DATA/MovieTriples_Dataset/"

train, valid, test = load_mt(path=mt_path, split=True, trim=200)
idx_to_w, w_to_idx, voc_size, freqs = get_mt_voc(path=mt_path, train_len=len(train))

In [4]:
word2vec_embs, word2vec_embs_mask = get_w2v_embs(path=mt_path)

w2v_train_mask = np.where(word2vec_embs_mask[:,0] == 1)[0]

In [5]:
hred_net = HRED(voc_size=voc_size,
                emb_size=300,
                lv1_rec_size=300, 
                lv2_rec_size=300, 
                out_emb_size=300, 
                num_sampled=200,
                ssoft_probs=freqs,
                emb_init=word2vec_embs)

Building the model...
Compiling theano functions...
Building a network for generating...
Done


In [6]:
hred_net.train_one_epoch(train, 60)

Done 10 batches in 1.89s	training loss:	7.119950
Done 20 batches in 3.92s	training loss:	6.173208
Done 30 batches in 6.17s	training loss:	5.686749
Done 40 batches in 8.17s	training loss:	5.410087
Done 50 batches in 10.29s	training loss:	5.226654
Done 60 batches in 12.06s	training loss:	5.082598
Done 70 batches in 14.04s	training loss:	4.958826
Done 80 batches in 16.05s	training loss:	4.872521
Done 90 batches in 17.68s	training loss:	4.799309
Done 100 batches in 19.47s	training loss:	4.737168
Done 110 batches in 21.20s	training loss:	4.679248
Done 120 batches in 23.17s	training loss:	4.638855
Done 130 batches in 25.62s	training loss:	4.600006
Done 140 batches in 27.42s	training loss:	4.560938
Done 150 batches in 29.69s	training loss:	4.524415
Done 160 batches in 31.64s	training loss:	4.494946
Done 170 batches in 33.95s	training loss:	4.464296
Done 180 batches in 36.23s	training loss:	4.437929
Done 190 batches in 38.21s	training loss:	4.413754
Done 200 batches in 39.76s	training loss:	4.

3.5235153449672456

In [7]:
hred_net.validate(valid, 30)

Done 100 batches in 4.70s
Done 200 batches in 9.61s
Done 300 batches in 14.72s
Done 400 batches in 19.20s
Done 500 batches in 23.82s
Done 600 batches in 28.30s
Done 700 batches in 32.87s
Done 800 batches in 37.37s
Done 900 batches in 42.25s
Done 1000 batches in 47.36s
Done 1100 batches in 52.21s
Done 1200 batches in 57.24s
Done 1300 batches in 62.33s
Done 1400 batches in 66.94s
Done 1500 batches in 71.54s
Done 1600 batches in 76.18s
Done 1700 batches in 80.71s
Done 1800 batches in 85.81s
Done 1900 batches in 90.87s
Done 2000 batches in 95.38s
Done 2100 batches in 100.35s
Done 2200 batches in 105.36s
Done 2300 batches in 110.49s
Done 2400 batches in 115.22s


3.5063306987542759

In [8]:
'''full softmax, bs=30'''
# train, 1 dir, 1 epoch: 3.485554076321884
# val: 3.455356876018342

# train, 2 dir, concat, 1 epoch: 3.4864403798772239
# val: 3.4579001751897063

# train, 2 dir, L2 + concat, 1 epoch: 3.4881669768474675
# val: 3.4584704095551695
# training time: ~4700s

'''sampled softmax'''
# bs=30
# train, 2 dir, L2 + concat, 1 epoch: 3.486180601246621
# val: 3.4811877499289308
# training time: ~2300s

# bs=60
# train, 2 dir, L2 + concat, 1 epoch: 3.5235153449672456
# val: 3.5063306987542759
# training time: ~1900s